In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# to increase no. of rows and column visibility in outputs
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
train= pd.read_json("/content/drive/My Drive/github bug data/embold_train.json").reset_index(drop=True)
test=pd.read_json("/content/drive/My Drive/github bug data/embold_test.json").reset_index(drop=True)
train_ex=pd.read_json("/content/drive/My Drive/github bug data/embold_train_extra.json").reset_index(drop=True)
# sub=pd.read_csv("../input/github-bug-prediction/sample submission.csv")

In [5]:
train.head()
test.head()
# train_ex.head()
train=pd.concat([train,train_ex],axis=0)
# sub.head()

,title,body,label
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0


,title,body
0,config question path-specific environment var...,issue description or question\r \r hey @artemg...
1,crash indien vol,de simulator crasht als hij vol zit
2,unable to mine rocks,"sarkasmo starting today, when i hit enter act..."
3,not all whitelists are processed,create following rules... order of creation is...
4,add ctx menu for idafree 70 and idafree 5,"associated with .dll, .dll_, .exe, .exe_, .sc,..."


In [6]:
train['text']=train['title']+' '+train['body']
test['text']=test['title']+' '+test['body']

In [7]:
import re
def rem_links(text):
  text_list=text.split()
  text_list=[w for w in text_list if not (w.startswith('http') or '@' in w)]
  return ' '.join(text_list)

In [8]:
train['text']=train['text'].apply(rem_links)
test['text']=test['text'].apply(rem_links)

In [9]:
# import re
# # #cleaning text
# # train['body']=train['body'].str.replace('_',' ')
# # test['body']=test['body'].str.replace('_',' ')
train['text']=train['text'].str.replace(r'\\r',r'')
test['text']=test['text'].str.replace(r'\\r',r'')
train.loc[2,'text']

2    auto update feature hi,  great job so far, ! :...
2    add multilang support for timeago.js currently...
Name: text, dtype: object

In [10]:
train=train.drop(['title','body'],axis=1)
test=test.drop(['title','body'],axis=1)

In [11]:
!pip -q install transformers

     |████████████████████████████████| 1.1MB 5.7MB/s 
     |████████████████████████████████| 890kB 43.4MB/s 
     |████████████████████████████████| 1.1MB 44.2MB/s 
     |████████████████████████████████| 3.0MB 52.2MB/s 


In [12]:
import torch 
from transformers import RobertaTokenizer,RobertaModel,get_linear_schedule_with_warmup
import transformers 
import torch
import torch.nn as nn 
import pandas as pd 
import numpy as np 
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from sklearn.model_selection import StratifiedKFold,train_test_split
import os 
import random

In [13]:
#train,_,_,_=train_test_split(train,train['label'],test_size=.75,shuffle=True,stratify=train['label'],random_state=160639)

In [14]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [15]:
CONFIG = {
    'MAX_LEN':64,
    'TRAIN_BATCH_SIZE':32,
    'VALID_BATCH_SIZE':32,
    'EPOCHS':3,
    'TOKENIZER':RobertaTokenizer.from_pretrained('roberta-base',lowercase=True,truncation=True)
}

In [16]:
import random
from random import randint
import numpy as np
SEED_VAL  = 1000
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
  random.seed(SEED_VAL)
  np.random.seed(SEED_VAL)
  torch.manual_seed(SEED_VAL)
  torch.cuda.manual_seed_all(SEED_VAL)
  os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
  torch.backends.cudnn.deterministic = True

In [17]:
class CustomRoberta(nn.Module):
    def __init__(self):
        super(CustomRoberta, self).__init__()
        self.num_labels = 3
        self.roberta = transformers.RobertaModel.from_pretrained("roberta-base", output_hidden_states=False, num_labels=self.num_labels)
        self.dropout = nn.Dropout(p=0.2)
        self.classifier = nn.Linear(768, self.num_labels)

    def forward(self,
                input_ids=None,
                attention_mask=None,
                position_ids=None,
                head_mask=None,
                inputs_embeds=None):

        _, o2 = self.roberta(input_ids,
                               attention_mask=attention_mask,
                               position_ids=position_ids,
                               head_mask=head_mask,
                               inputs_embeds=inputs_embeds)
        o2 = self.dropout(o2)
        logits = self.classifier(o2)       
        outputs = logits
        return outputs

In [18]:
class RobertaDataset:
  def __init__(self,tweet,target=None,task='train'):
    self.tweet= tweet
    self.target = target
    self.tokenizer = CONFIG['TOKENIZER']
    self.max_len = CONFIG['MAX_LEN']
    self.task = task
  
  def __len__(self):
    return len(self.tweet)

  def __getitem__(self,item):
    tweet = str(self.tweet[item])
    tweet = ' '.join(tweet.split())


    inputs = self.tokenizer.encode_plus(tweet,
                                        max_length=self.max_len,
                                        pad_to_max_length=True,
                                        add_special_tokens=True,
                                        truncation=True)
    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    


    to_return= {
        'ids':torch.tensor(ids,dtype=torch.long),
        'mask':torch.tensor(mask,dtype=torch.long),
    }
    if (self.task=='train'):

      to_return.update({'target':torch.tensor(self.target[item])})

    return to_return

In [19]:
def loss_fn(outputs,targets):
  criterion =  nn.CrossEntropyLoss()
  return criterion(outputs,targets)

In [20]:
def train_fn(data_loader,model,optimizer,device,sc=None):
  model.train()
  tot_loss = 0
  for bi, d in enumerate(data_loader):
    ids = d['ids']
    mask = d['mask']
    targets = d['target']

    #send them to cuda gpu 
    ids = ids.to(device,dtype=torch.long)
    mask = mask.to(device,dtype=torch.long)
   
    targets = targets.to(device,dtype=torch.long)
    
    optimizer.zero_grad()

    outputs = model(
        ids,
        mask,
    )
    
    loss = loss_fn(outputs,targets)
    tot_loss += loss.item()
    loss.backward()
    optimizer.step()
    if sc:
      sc.step()
  
  print("Training loss for this epoch: ",tot_loss/len(data_loader))

In [21]:
def eval_fn(data_loader,model,device):
  model.eval()
  fin_targets = []
  fin_outputs =[]
  tot_loss = 0
  with torch.no_grad():
    for bi, d in enumerate(data_loader):
      ids = d['ids']
      mask = d['mask']
      
      targets = d['target']

      #send them to cuda gpu 
      ids = ids.to(device,dtype=torch.long)
      mask = mask.to(device,dtype=torch.long)
      
     
      targets = targets.to(device,dtype=torch.long)
      
      

      outputs = model(
          ids,
          mask
      )

      loss = loss_fn(outputs,targets)
      tot_loss+=loss.item()
      fin_targets.extend(targets.cpu().detach().numpy())
      fin_outputs.extend(torch.nn.functional.softmax(outputs).cpu().detach().numpy())
  return fin_outputs,fin_targets,tot_loss/(len(data_loader))

In [22]:
#test = pd.read_csv('test.csv')
test_dataset = RobertaDataset(
    tweet=test.text.values,
    task='test'
)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=CONFIG['TRAIN_BATCH_SIZE'],
    num_workers=4
)

In [23]:
def predict_fn(model):
  fin_outputs = []
  with torch.no_grad():
    for bi, d in enumerate(test_data_loader):
      ids = d['ids']
      mask = d['mask']
  
      #send them to cuda gpu 
      ids = ids.to(device,dtype=torch.long)
      mask = mask.to(device,dtype=torch.long)
      
     
      outputs = model(
          ids,
          mask
      )
      fin_outputs.append(torch.nn.functional.softmax(outputs).cpu().detach().numpy())
      alls = np.vstack(fin_outputs)

  return alls

In [24]:
def run_folds():
    total_folds=1
    all_preds = []
    losses = []
    seed_all(SEED_VAL)
    dfx = train.fillna("none")
    dfx['label'] = dfx['label'].factorize()[0]
    df_train,df_valid,_,_=train_test_split(dfx,dfx['label'],test_size=.17,shuffle=True,stratify=dfx['label'],random_state=160639)
    for i in range(1):
    #fold=StratifiedKFold(n_splits=total_folds, shuffle=True)
    #for i,(train_index, test_index) in enumerate(fold.split(dfx,dfx['label'])):
     # print(f'FOLD {i+1}/{total_folds}')
      #df_train = dfx.iloc[train_index]
      #df_valid = dfx.iloc[test_index]

      train_dataset =RobertaDataset(
          tweet=df_train.text.values,
          target=df_train.label.values,
          task='train'
      )

      train_data_loader = torch.utils.data.DataLoader(
          train_dataset,
          batch_size=CONFIG['TRAIN_BATCH_SIZE'],
          num_workers=4
      )

      valid_dataset =RobertaDataset(
          tweet=df_valid.text.values,
          target=df_valid.label.values,
          task='train'
      )

      valid_data_loader = torch.utils.data.DataLoader(
          valid_dataset,
          batch_size=CONFIG['TRAIN_BATCH_SIZE'],
          num_workers=1
      )

      device = torch.device("cuda")
      model = CustomRoberta()
      model.to(device)
      
      param_optimizer = list(model.named_parameters())
      no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
      optimizer_parameters = [
          {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
          {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
      ]

      num_train_steps = int(len(df_train) / CONFIG['TRAIN_BATCH_SIZE'] * CONFIG['EPOCHS'])
      optimizer = AdamW(optimizer_parameters, lr=1e-5)
      
      #scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=num_train_steps)


      best_accuracy = 0
      for epoch in range(CONFIG['EPOCHS']):
          print("----------------EPOCH "+str(epoch+1)+"---------------------")
          train_fn(train_data_loader, model, optimizer, device#scheduler
                  )
          outputs,targets,losss = eval_fn(valid_data_loader ,model, device)
          print("LOSS for this Epoc on val: ",losss)
      losses.append(losss)
      fold_preds = predict_fn(model)
      all_preds.append(fold_preds)
    print("mean losses over all folds: ",np.mean(losses))
    return  all_preds

In [25]:
preds = run_folds()


----------------EPOCH 1---------------------


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Training loss for this epoch:  0.4661143212218535


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


LOSS for this Epoc on val:  0.4294603568478789
----------------EPOCH 2---------------------


...

Training loss for this epoch:  0.3893860901180576


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


LOSS for this Epoc on val:  0.43397654039884204


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

mean losses over all folds:  0.43397654039884204


In [26]:
preds_1 = np.mean(preds,axis=0)
preds_1.shape

(30000, 3)

In [31]:
train['label'].factorize()

(array([0, 1, 0, ..., 1, 0, 1]), Int64Index([1, 0, 2], dtype='int64'))

In [34]:
preds_1[:, [1, 0]] = preds_1[:, [0, 1]] 

In [36]:
predictions=preds_1.argmax(axis=1)
submission_df = pd.DataFrame(predictions, columns=['label'])
# #write a .csv file for submission
submission_df.to_csv('bert_trial.csv', index=False)